# ASL Detection Training

In [ ]:
import torch

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("WARNING: No GPU detected")
    print("Go to Runtime -> Change runtime type -> Select GPU")

In [ ]:
#dependencies
!pip install -q ultralytics opencv-python-headless
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import yaml
import shutil

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive_dataset_path = '/content/drive/MyDrive/American Sign Language Letters.v6-raw.yolov8'
local_dataset_path = '/content/data/American Sign Language Letters.v6-raw.yolov8'

!mkdir -p /content/data
!cp -r "{drive_dataset_path}" /content/data/

print("Done")

In [ ]:
#verify dataset
dataset_path = Path(local_dataset_path)
data_yaml = dataset_path / 'data.yaml'

if data_yaml.exists():
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    
    print(f"Classes: {data_config.get('nc')}")
    print(f"Names: {data_config.get('names')}")
    
    for split in ['train', 'valid', 'test']:
        split_path = dataset_path / split / 'images'
        if split_path.exists():
            img_count = len(list(split_path.glob('*.jpg'))) + len(list(split_path.glob('*.png')))
            print(f"{split}: {img_count} images")
else:
    print("ERROR: data.yaml not found")
    print(f"Check path: {drive_dataset_path}")

Fix paths

In [ ]:
with open(data_yaml, 'r') as f:
    config = yaml.safe_load(f)

config['path'] = str(dataset_path)
config['train'] = 'train/images'
config['val'] = 'valid/images'
config['test'] = 'test/images'

with open(data_yaml, 'w') as f:
    yaml.dump(config, f)

print(f"Dataset path: {config['path']}")

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
device = '0' if torch.cuda.is_available() else 'cpu'
print(f"Device: {'GPU' if device == '0' else 'CPU'}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

results = model.train(
    data=str(data_yaml),
    epochs=50,
    imgsz=640,
    batch=32,
    name='asl_detector',
    patience=10,
    save=True,
    project='/content/output',
    device=device,
    plots=True,
    cache=True
)

print("Training complete")

Validate

In [ ]:
best_model = YOLO('/content/output/asl_detector/weights/best.pt')
metrics = best_model.val(device=device)

print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")

Test samples

In [ ]:
test_images_path = dataset_path / 'test' / 'images'
test_images = list(test_images_path.glob('*.jpg'))[:5]

results = best_model.predict(
    source=test_images,
    save=True,
    project='/content/output/predictions',
    name='test_samples',
    conf=0.25
)

fig, axes = plt.subplots(1, len(test_images), figsize=(15, 3))
if len(test_images) == 1:
    axes = [axes]

for idx, result in enumerate(results):
    img_with_boxes = result.plot()
    img_rgb = cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB)
    axes[idx].imshow(img_rgb)
    axes[idx].axis('off')
    axes[idx].set_title(f'Sample {idx+1}')

plt.tight_layout()
plt.show()

Download

In [ ]:
from google.colab import files

drive_output_path = '/content/drive/MyDrive/asl_model_trained'
!mkdir -p "{drive_output_path}"
!cp /content/output/asl_detector/weights/best.pt "{drive_output_path}/asl_best.pt"

print(f"Saved to Drive: {drive_output_path}/asl_best.pt")

files.download('/content/output/asl_detector/weights/best.pt')
